In [ ]:
from google.colab import drive
import os
drive.mount('carga')

Mounted at carga


In [ ]:
#CUSTOM_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'   
CUSTOM_MODEL_NAME = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18'

IMAGE_PATH = '/content/carga/MyDrive/workspace/images'
ANNOTATION_PATH = '/content/carga/MyDrive/workspace/annotations' 
OUTPUT_PATH = '/content/carga/MyDrive/workspace/tf1/35/'+ CUSTOM_MODEL_NAME + '/output'
EXPORT_PATH = '/content/carga/MyDrive/workspace/tf1/35/'+ CUSTOM_MODEL_NAME + '/export'

In [ ]:
# Importar tensorflow 1
%tensorflow_version 1.x
!pip install tf_slim
!pip show tensorflow

TensorFlow is already loaded. Please restart the runtime to change versions.
     |████████████████████████████████| 358kB 8.6MB/s 
Name: tensorflow
Version: 2.4.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: absl-py, h5py, astunparse, numpy, opt-einsum, tensorboard, protobuf, flatbuffers, gast, wheel, tensorflow-estimator, grpcio, wrapt, typing-extensions, keras-preprocessing, termcolor, six, google-pasta
Required-by: fancyimpute


In [ ]:
# Clonar la API de deteccion de objetos
!apt-get install protobuf-compiler
!git clone https://github.com/tensorflow/models.git

import os
os.environ['PYTHONPATH'] += ':/content/models/research/'
os.environ['PYTHONPATH'] += ':/content/models/research/slim/'
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection/utils/'
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection'

%cd models/research
# Compilar las dependencias
!protoc object_detection/protos/*.proto --python_out=.
# Instalar la API
!cp object_detection/packages/tf1/setup.py .
!python -m pip install .
# Correr un test
!python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
Cloning into 'models'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 52095 (delta 4), reused 8 (delta 0), pack-reused 52077
Receiving objects: 100% (52095/52095), 569.10 MiB | 43.04 MiB/s, done.
Resolving deltas: 100% (35556/35556), done.
/content/models/research
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1619116 sha256=acd86238b8d55cccc0e3b07dd78b1d8e5084ca9734c1325a265e1e4fc52c30cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-8y9mxn9g/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
2021-02-22 02:36:00.973166: I tenso

##**Convert**

In [ ]:
# Convertir el modelo

!tflite_convert \
  --output_file=$EXPORT_PATH/tflite/model.tflite \
  --saved_model_dir=$EXPORT_PATH/export_graph_tf1/saved_model\
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays="normalized_input_image_tensor" \
  --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,320,320,3 \
  --allow_custom_ops

2021-02-20 04:15:33.722202: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-02-20 04:15:33.817333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-02-20 04:15:33.817920: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-02-20 04:15:33.818177: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-02-20 04:15:34.049288: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-02-20 04:15:34.167060: I tensorflow/stream_executor/plat

In [ ]:
interpreter = tf.lite.Interpreter(tflite_model)

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: input
shape: [  1 320 320   3]
type: <class 'numpy.float32'>

== Output details ==
name: Identity
shape: []
type: <class 'numpy.float32'>
